In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/home/aniket/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
train_subset = 100000
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 14.845035
Training accuracy: 12.5%
Validation accuracy: 15.1%
Loss at step 100: 2.392921
Training accuracy: 71.3%
Validation accuracy: 71.5%
Loss at step 200: 2.002017
Training accuracy: 73.9%
Validation accuracy: 73.9%
Loss at step 300: 1.789577
Training accuracy: 74.9%
Validation accuracy: 74.8%
Loss at step 400: 1.643127
Training accuracy: 75.5%
Validation accuracy: 75.5%
Loss at step 500: 1.532835
Training accuracy: 76.0%
Validation accuracy: 76.1%
Loss at step 600: 1.445376
Training accuracy: 76.3%
Validation accuracy: 76.4%
Loss at step 700: 1.373679
Training accuracy: 76.6%
Validation accuracy: 76.7%
Loss at step 800: 1.313543
Training accuracy: 76.9%
Validation accuracy: 77.1%
Test accuracy: 83.5%


In [13]:
batch_size = 128
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 6001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    offset = 0
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_dataset = train_dataset[offset:offset+batch_size,:]
        batch_labels = train_labels[offset:offset+batch_size,:]
        feed_dict = {tf_train_dataset : batch_dataset, tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss, train_prediction], feed_dict = feed_dict)
        if step%500 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 19.106249
Minibatch accuracy: 7.0%
Validation accuracy: 9.5%
Loss at step 500: 1.543135
Minibatch accuracy: 72.7%
Validation accuracy: 76.3%
Loss at step 1000: 1.686170
Minibatch accuracy: 72.7%
Validation accuracy: 76.8%
Loss at step 1500: 1.379344
Minibatch accuracy: 79.7%
Validation accuracy: 77.9%
Loss at step 2000: 1.310466
Minibatch accuracy: 78.1%
Validation accuracy: 78.2%
Loss at step 2500: 1.270024
Minibatch accuracy: 70.3%
Validation accuracy: 79.0%
Loss at step 3000: 1.019339
Minibatch accuracy: 76.6%
Validation accuracy: 79.7%
Loss at step 3500: 0.951776
Minibatch accuracy: 82.0%
Validation accuracy: 79.9%
Loss at step 4000: 0.916080
Minibatch accuracy: 79.7%
Validation accuracy: 79.9%
Loss at step 4500: 0.990451
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%
Loss at step 5000: 0.902164
Minibatch accuracy: 78.1%
Validation accuracy: 79.7%
Loss at step 5500: 0.454673
Minibatch accuracy: 86.7%
Validation accuracy: 80.4%
Loss at step 6000: 0.

In [15]:
def create_hl(ip_node, op_node):
    weight = tf.Variable(tf.truncated_normal([ip_node,op_node]))
    biases = tf.Variable(tf.zeros([op_node]))
    layer = {'weights': weight, 'biases': biases}
    return layer

batch_size = 128
hl1_node = 1024
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    l1 = create_hl(image_size*image_size,hl1_node)
    hl1 = create_hl(hl1_node,num_labels)
    
    hlin_logits = tf.nn.relu(tf.matmul(tf_train_dataset, l1['weights']) + l1['biases'])
    hlout_logits = tf.matmul(hlin_logits, hl1['weights']) + hl1['biases']
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=hlout_logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    train_prediction = tf.nn.softmax(hlout_logits)

    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, l1['weights']) + l1['biases']), 
                                               hl1['weights']) + hl1['biases'])
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, l1['weights']) + l1['biases']), 
                                              hl1['weights']) + hl1['biases'])

In [16]:
num_steps = 6001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    offset = 0
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0] - batch_size)
        batch_dataset = train_dataset[offset:offset+batch_size,:]
        batch_labels = train_labels[offset:offset+batch_size,:]
        feed_dict = {tf_train_dataset : batch_dataset, tf_train_labels : batch_labels}
        _,l,predictions = session.run([optimizer,loss, train_prediction], feed_dict = feed_dict)
        if step%500 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 259.781952
Minibatch accuracy: 18.8%
Validation accuracy: 29.7%
Loss at step 500: 12.677946
Minibatch accuracy: 78.9%
Validation accuracy: 80.9%
Loss at step 1000: 16.398695
Minibatch accuracy: 75.8%
Validation accuracy: 80.8%
Loss at step 1500: 8.203747
Minibatch accuracy: 77.3%
Validation accuracy: 81.8%
Loss at step 2000: 14.037598
Minibatch accuracy: 74.2%
Validation accuracy: 79.3%
Loss at step 2500: 5.395738
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Loss at step 3000: 4.396937
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Loss at step 3500: 3.429407
Minibatch accuracy: 85.9%
Validation accuracy: 83.0%
Loss at step 4000: 5.908369
Minibatch accuracy: 83.6%
Validation accuracy: 82.3%
Loss at step 4500: 5.183830
Minibatch accuracy: 85.2%
Validation accuracy: 82.6%
Loss at step 5000: 5.667267
Minibatch accuracy: 84.4%
Validation accuracy: 83.2%
Loss at step 5500: 4.217988
Minibatch accuracy: 86.7%
Validation accuracy: 83.0%
Loss at step 60